In [1]:
from pyscf import scf,gto
import numpy as np
import inspect
from FcMole import FcM
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf  #### very important
from pyscf.hessian import rhf as hrhf # without those two mf.Gradients() and mf.Hessian() don't work
def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    if len(dL)>1:
        for i in range(1,len(dL)):
            mol.set_rinv_orig_(mol.atom_coords()[i])
            dV+=mol.intor('int1e_rinv')*dL[i]
    return -dV

First compare gradient analytical from fcm with the one via finite differences on geometry


In [2]:
mol1=FcM(fcs=[.001,-.001],atom="C 0 0 0; O 0 0 1.8",unit="Bohr",basis="STO-3G")
mf1=scf.RHF(mol1)
mf1.scf(dm0=mf1.init_guess_by_1e())
g1=mf1.Gradients()
g1.run()

converged SCF energy = -111.05722603438
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000     1.0869705326
1 O    -0.0000000000    -0.0000000000    -1.0869705326
----------------------------------------------


In [3]:
mol0=gto.M(atom="C 0 0 0; O 0 0 1.8",unit="Bohr",basis="STO-3G")
mf0=scf.RHF(mol0)
mf0.scf()
g0=mf0.Gradients()
g0.kernel()

converged SCF energy = -111.064463936466
--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0000000000     0.0000000000     1.0871732099
1 O     0.0000000000    -0.0000000000    -1.0871732099
----------------------------------------------


array([[-9.33660854e-17,  2.66546202e-16,  1.08717321e+00],
       [ 9.33660854e-17, -2.66546202e-16, -1.08717321e+00]])

In [4]:
mf0.mol

In [5]:
from alch_deriv import alch_deriv

The formula for the gradient is stated in Pople's article (Eq.21) as: 
$$ \frac{\partial E}{\partial x}= \sum_{\mu\nu}P_{\mu\nu}\frac{\partial H_{\mu\nu}}{\partial x}+\frac{1}{2}\sum_{\mu\nu\lambda\sigma}
P_{\mu\nu}P_{\lambda\sigma}\frac{\partial}{\partial x}(\mu \lambda | | \nu\sigma)+\frac{\partial V_{nuc}}{\partial x} 
-\sum_{\mu\nu}W_{\mu\nu}\frac{\partial S_{\mu\nu}}{\partial x}
$$
$W$ is an energy weighted density matrix:
$$ W_{\mu\nu}= \sum_i ^{mo.occ.} c_{\mu i} \epsilon_i    c_{\nu i}^\dagger = S^{-1}FCOC^\dagger=S^{-1}FP
$$

In [6]:
(U,dP)=alch_deriv(mf0,dL=[[0,1],[.001,-.001]])
P=mf0.make_rdm1()
P1=mf1.make_rdm1()

[[0, 1], [0.001, -0.001]]


First piece:
$$ \frac{\partial}{\partial Z} ( P \frac{\partial H^{(1)}}{\partial x})= 
\frac{\partial P}{\partial Z}\frac{\partial H^{(1)}}{\partial x}+ P \frac{\partial^2 H^{(1)}}{\partial x \partial Z}
$$
$\frac{\partial^2 H^{(1)}}{\partial x \partial Z}$ is trivially $\frac{\partial H^{(1)}}{\partial x}$ divided by the atom charge

In [7]:
ga=np.zeros((2,3))
ga[0]+=np.einsum('xij,ij->x', g0.hcore_generator()(0),dP)       #dP/dZ* dH/dx
ga[1]+=np.einsum('xij,ij->x', g0.hcore_generator()(1),dP)
print(ga)
ga[0]+=np.einsum('xij,ij->x', g1.hcore_generator()(0)-g0.hcore_generator()(0),P)   #P* ddH/dZdx
ga[1]+=np.einsum('xij,ij->x', g1.hcore_generator()(1)-g0.hcore_generator()(1),P)
print(ga)

[[ 1.05677455e-18  2.77992946e-18 -4.42827506e-03]
 [-1.05677455e-18 -2.77992946e-18  4.42827506e-03]]
[[ 7.13794964e-19  2.82054211e-18 -5.24933658e-03]
 [-7.13794964e-19 -2.82054211e-18  5.24933658e-03]]


In [8]:
comp1=np.zeros((2,3,10,10))
comp1[0]=g1.hcore_generator()(0)-g0.hcore_generator()(0)
comp1[1]=g1.hcore_generator()(1)-g0.hcore_generator()(1)

In [9]:
comp2=np.zeros((2,3,10,10)) 
dL=[.001,-.001]
for atm_id in [0,1]:
    with mol0.with_rinv_at_nucleus(atm_id):
        vrinv = -mol0.intor('int1e_iprinv', comp=3)
    shl0, shl1, p0, p1 = mol0.aoslice_by_atom()[atm_id]
    vrinv*=dL[atm_id]
    vrinv[:,p0:p1] += (g1.get_hcore()-g0.get_hcore())[:,p0:p1]  #bearbeiten
    vrinv += vrinv.transpose(0,2,1)
    comp2[atm_id]=vrinv

In [10]:
np.allclose(comp1,comp2)   #the difference lyes in  g.get_hcore()

True

g0.get_hcore() is the integral $<\chi_\mu |\nabla_r \hat{H}^{(1)}|\chi_\nu> $ is composed by two parts: the first refered to the kintic energy operator which is alchemy invariant, the second which has to be computed is refered to the nuclear electron attraction. <br>
To compute this we use moleintor.getints() using as arguments a mol environment (mol._env) with the added fractional charges and a mol._atm desription that show fractional charges.
Not forget to put a minus sign !!!! 

In [11]:
nao=mol0.nao
NUC_FRAC_CHARGE=gto.mole.NUC_FRAC_CHARGE
NUC_MOD_OF=gto.mole.NUC_MOD_OF
PTR_FRAC_CHARGE=gto.mole.PTR_FRAC_CHARGE
denv=mol0._env.copy()
datm=mol0._atm.copy()
fcs=[.001,-.001]
datm[:,NUC_MOD_OF] = NUC_FRAC_CHARGE
for i in range (mol0.natm):
    denv[datm[i,PTR_FRAC_CHARGE]]=fcs[i]
dH1=-gto.moleintor.getints('int1e_ipnuc_sph',datm,mol0._bas,denv, None,3,0,'s1')   #minus sign !

In [12]:
comp2=np.zeros((2,3,nao,nao))
dL=[.001,-.001]
for atm_id in [0,1]:
    with mol0.with_rinv_at_nucleus(atm_id):
        vrinv = -mol0.intor('int1e_iprinv', comp=3)
    shl0, shl1, p0, p1 = mol0.aoslice_by_atom()[atm_id]
    vrinv*=dL[atm_id]
    vrinv[:,p0:p1] += dH1[:,p0:p1]  #bearbeiten
    vrinv += vrinv.transpose(0,2,1)
    comp2[atm_id]=vrinv
np.allclose(comp1,comp2)

True

In [13]:
fdg=np.zeros((2,3))   #finite difference part 1 gradient
fdg[0]+=np.einsum('xij,ij->x', g1.hcore_generator()(0),mf1.make_rdm1())
fdg[0]-=np.einsum('xij,ij->x', g0.hcore_generator()(0),mf0.make_rdm1())
fdg[1]+=np.einsum('xij,ij->x', g1.hcore_generator()(1),mf1.make_rdm1())
fdg[1]-=np.einsum('xij,ij->x', g0.hcore_generator()(1),mf0.make_rdm1())
fdg

array([[-2.21452354e-15,  3.68044216e-14, -5.24349964e-03],
       [ 2.21452354e-15, -3.68044216e-14,  5.24349964e-03]])

In [14]:
ga_1=np.zeros((2,3))     # analytical gradient _part 1 
ga_1[0]+=np.einsum('xij,ij->x', g0.hcore_generator()(0),dP)
ga_1[1]+=np.einsum('xij,ij->x', g0.hcore_generator()(1),dP)
#print(ga_1)
ga_1[0]+=np.einsum('xij,ij->x',  comp2[0],P)
ga_1[1]+=np.einsum('xij,ij->x', comp2[1],P)
print(ga_1)

[[ 7.13794964e-19  2.82054211e-18 -5.24933658e-03]
 [-7.13794964e-19 -2.82054211e-18  5.24933658e-03]]


In [15]:
from aaff import g1 as g1fun
g1fun(mol0,dP,P,[.001,-.001],g0)

array([[ 7.13794964e-19,  2.82054211e-18, -5.24933658e-03],
       [-7.13794964e-19, -2.82054211e-18,  5.24933658e-03]])

# Second piece:
$$\frac{\partial}{\partial Z} (P_{\mu\nu}P_{\lambda\sigma}\frac{\partial}{\partial x}(\mu \lambda | | \nu\sigma) )$$
here the two electron integral is invariant to alchemy, therefore is sufficient insert the density matrix derivative $dP$ in the following exression. 

In [16]:
#for the ref molecule
aoslices = mol0.aoslice_by_atom()
g2e_part2_0=np.zeros((2,3))
for ia in [0,1]:
    p0, p1 = aoslices [ia,2:]
    vhf = g0.get_veff(mol0, P)
    g2e_part2_0[ia]=(np.einsum('xij,ij->x', vhf[:,p0:p1], P[p0:p1]) * 2)        #   P (Pd/dx(ml||ns))
g2e_part2_0 

array([[-9.26734490e-15, -3.57864553e-15,  1.17925050e+01],
       [ 9.26734490e-15,  3.57864553e-15, -1.17925050e+01]])

In [17]:
aoslices = mol1.aoslice_by_atom()
g2e_part2_1=np.zeros((2,3))
for ia in [0,1]:
    p0, p1 = aoslices [ia,2:]
    vhf = g1.get_veff(mol1, P1)
    g2e_part2_1[ia]=(np.einsum('xij,ij->x', vhf[:,p0:p1], P1[p0:p1]) * 2) 
g2e_part2_1

array([[-7.75847842e-15, -3.63920220e-14,  1.17969070e+01],
       [ 7.75847842e-15,  3.63920220e-14, -1.17969070e+01]])

In [18]:
g_fd_p2=g2e_part2_1-g2e_part2_0

In [19]:
#check the invariance:
np.allclose(g0.get_veff(mol0, P),g1.get_veff(mol1, P1))

False

In [20]:
print(np.allclose(g0.get_veff(mol0, P1),g1.get_veff(mol1, P1))) #update P
print(np.allclose(g0.get_veff(mol0, P+dP),g1.get_veff(mol1, P1))) #update P

True
True


In [21]:
aoslices = mol0.aoslice_by_atom()
ga_2=np.zeros((2,3))
for ia in range(mol0.natm):
    p0, p1 = aoslices [ia,2:]
    vhf = g0.get_veff(mol0, P)
    vhf_1 = g0.get_veff(mol0, P+dP)
    ga_2[ia]=(np.einsum('xij,ij->x', vhf[:,p0:p1], dP[p0:p1]) * 2)
    ga_2[ia]+=(np.einsum('xij,ij->x',vhf_1[:,p0:p1]-vhf[:,p0:p1], P[p0:p1]) * 2)
ga_2,g_fd_p2

(array([[-8.71323925e-19, -2.48382771e-18,  4.40672861e-03],
        [ 8.71323925e-19,  2.48382771e-18, -4.40672861e-03]]),
 array([[ 1.50886648e-15, -3.28133765e-14,  4.40195062e-03],
        [-1.50886648e-15,  3.28133765e-14, -4.40195062e-03]]))

In [22]:
np.allclose(ga_2,g_fd_p2,atol=1e-3)

True

In [23]:
from aaff import g2 as g2fun
g2fun(mol0,dP,P,[.001,-.001],g0)

array([[-8.71323925e-19, -2.48382771e-18,  4.40672861e-03],
       [ 8.71323925e-19,  2.48382771e-18, -4.40672861e-03]])

# Third piece:
$$-\sum_{\mu\nu}W_{\mu\nu}\frac{\partial S_{\mu\nu}}{\partial x}
$$
Luckily $S$ is invariant in alchemy, therefore the different in gradient is just:$$
-\sum_{\mu\nu}\frac{\partial W_{\mu\nu}}{\partial Z}\frac{\partial S_{\mu\nu}}{\partial x}
$$
### Obtaining derivatives of W
$$W=  \sum_i ^{mo.occ.} \epsilon_i C_{\mu i} C_{\nu i}^\dagger 
$$

$$ \frac{\partial W}{\partial Z_I}= \sum_i ^{mo.occ.} \left( \epsilon_i (CU)_{\mu i} C_{\nu i}^\dagger + 
\epsilon_i C_{\mu i} (CU)^\dagger_{\nu i}   +\frac{\partial \epsilon_i}{\partial Z_I} C_{\mu i} C_{\nu i}^\dagger \right)$$


$$ S W= S C\epsilon C^\dagger=FCC^\dagger= FP $$
$$W=  S^{-1}FP $$

In [24]:
"""  THE CODE IN g.grad_elec()
dme0 = mf_grad.make_rdm1e(mo_energy, mo_coeff, mo_occ)        W
s1 = mf_grad.get_ovlp(mol)%autocall                         dS/dx
for k, ia in enumerate(atmlst):
    de[k] -= numpy.einsum('xij,ij->x', s1[:,p0:p1], dme0[p0:p1]) * 2        W dS/dx
"""
pass

In [25]:
#verify that s1 is invariant
s1=g0.get_ovlp(mol0)
np.allclose(g0.get_ovlp(mol0),g1.get_ovlp(mol1))

True

In [26]:
#at first by finite differences 
W1=g1.make_rdm1e()
W0=g0.make_rdm1e()
fd_dW=W1-W0

In [27]:
# W0 can be constructed via C@np.diag(o*e)@C.T
o=mf0.mo_occ
O=np.diag(o)
e=mf0.mo_energy
C=mf0.mo_coeff
S=mf0.get_ovlp()
F=mf0.get_fock()
np.allclose(C@np.diag(e*o)@C.T,W0)

True

In [28]:
#to derive this expression first get dC as 
dC=C@U
#to get d(e) we need to get the fock hamiltonian and than get the new eigenvalues 
g_ijkl=mol0.intor('int2e_sph')
dF2el=np.einsum('ijkl,kl->ij',g_ijkl,dP)-0.5*np.einsum('ijkl,jl->ik',g_ijkl,dP)
dV=DeltaV(mol0,[.001,-.001])

In [29]:
#try to get dF_2e from PySCF response function  
vresp = mf0.gen_response(hermi=1)
vresp(dP)[0],dF2el[0]
np.allclose(vresp(dP),dF2el)

True

In [30]:
np.allclose(mf1.get_fock()-F,dV+dF2el,atol=1e-5)

True

In [31]:
#try to get e in another way , using Roothans equations FC=SCe
print(np.allclose(np.linalg.inv(S)@F@(C),C@np.diag(e))) # S^-1 F C= C e 
print(np.allclose(np.linalg.inv(S)@F@P,W0))  #S^-1 F (C O C.T) = S^-1 F P = C e O C.T =W !!!! S^-1 F P=W

True
True


In [32]:
dW_a3=np.linalg.inv(S)@(F+dV+dF2el)@(P+dP)-W0
np.max(fd_dW-dW_a3),np.max(fd_dW)

(7.643944250901313e-07, 0.013093640938230067)

In [33]:
#W=S^-1 F P -> dW=S^-1(dF*P+F*dP )
dW_a4=np.linalg.inv(S)@((F@dP)+(dV+dF2el)@P)
dW_a4[0],fd_dW[0],dW_a3[0]

(array([-9.90595441e-03,  2.01647986e-03,  6.06044120e-20, -1.23440089e-19,
         4.43376150e-04,  1.54522483e-06, -7.89086878e-04, -1.33009991e-20,
         5.60582361e-20,  7.19199956e-05]),
 array([-9.90520069e-03,  2.01640002e-03, -1.12931817e-16,  8.52562751e-17,
         4.43437784e-04,  1.54815296e-06, -7.89109883e-04,  9.06330736e-18,
        -1.00331625e-16,  7.19010056e-05]),
 array([-9.90596508e-03,  2.01696748e-03,  4.98957259e-17, -2.16040272e-17,
         4.43936274e-04,  1.55719425e-06, -7.89614684e-04,  8.57120622e-17,
        -3.68940581e-17,  7.22175305e-05]))

In [34]:
W1_contr=np.zeros((2,3))
ga_3=np.zeros((2,3))
W0_contr=np.zeros((2,3))
for ia in [0,1]:
    p0, p1 = mol0.aoslice_by_atom() [ia,2:]
    W1_contr[ia] -= np.einsum('xij,ij->x', s1[:,p0:p1], W1[p0:p1]) * 2   # comparison
    ga_3[ia] -= np.einsum('xij,ij->x', s1[:,p0:p1], dW_a4[p0:p1]) * 2
    W0_contr[ia] -= np.einsum('xij,ij->x', s1[:,p0:p1], W0[p0:p1]) * 2   # comparison
(ga_3),W1_contr-W0_contr

(array([[ 2.53755553e-20,  8.16863184e-19,  2.24576423e-05],
        [-4.91904006e-19, -3.97695118e-18, -2.24547460e-05]]),
 array([[ 1.27664882e-15, -2.91678092e-15,  2.18963806e-05],
        [-1.27664882e-15,  2.91678092e-15, -2.18963806e-05]]))

In [35]:
#W=S^-1F P is the way, cause of the orbital rotation
S=mf0.get_ovlp()
F=mf0.get_fock()
g_ijkl=mol0.intor('int2e_sph')
dF2el=np.einsum('ijkl,kl->ij',g_ijkl,dP)-0.5*np.einsum('ijkl,jl->ik',g_ijkl,dP)
dV=DeltaV(mol0,[.001,-.001])
dW_a3=np.linalg.inv(S)@(F+dV+dF2el)@(P+dP)-W0

In [36]:
from aaff import g3 as g3fun
g3fun(mol0,dP,P,[.001,-.001],g0,vresp,F)

array([[ 2.53755553e-20,  8.16863184e-19,  2.24576423e-05],
       [-4.91904006e-19, -3.97695118e-18, -2.24547460e-05]])

##  FINAL COMPARISON FOR THE ELECTRONIC GRADIENT 

In [37]:
from aaff import aaff
ga_1+ga_2+ga_3,g1.grad_elec()-g0.grad_elec(),aaff(mf0,[.001,-.001],dP)

(array([[-1.32153405e-19,  1.15357759e-18, -8.20150325e-04],
        [-3.34375045e-19, -4.31366558e-18,  8.20153221e-04]]),
 array([[ 5.70991770e-16,  1.07426422e-15, -8.19652647e-04],
        [-5.70991770e-16, -1.07426422e-15,  8.19652647e-04]]),
 array([[-1.32153405e-19,  1.15357759e-18, -8.20150325e-04],
        [-3.34375045e-19, -4.31366558e-18,  8.20153221e-04]]))

## At the end the nuclear nuclear part 

In [38]:
"""def grad_nuc(mol, atmlst=None):
    gs = numpy.zeros((mol.natm,3))
    for j in range(mol.natm):
        q2 = mol.atom_charge(j)      <----------------------- derive here
        r2 = mol.atom_coord(j)
        for i in range(mol.natm):
            if i != j:
                q1 = mol.atom_charge(i)     <----------------------- and here 
                r1 = mol.atom_coord(i)      
                r = numpy.sqrt(numpy.dot(r1-r2,r1-r2))
                gs[j] -= q1 * q2 * (r2-r1) / r**3
    if atmlst is not None:
        gs = gs[atmlst]
    return gs
    """
pass

In [ ]:
def alc_differential_grad_nuc(mol,dL, atmlst=None):
    gs = np.zeros((mol.natm,3))
    for j in range(mol.natm):
        q2 =  mol.atom_charge(j) + dL[j]
        r2 = mol.atom_coord(j) 
        for i in range(mol.natm):
            if i != j:
                q1 = mol.atom_charge(i) +dL[i]
                r1 = mol.atom_coord(i)
                r = np.linalg.norm(r1-r2)
                gs[j] = (q1 * q2-mol.atom_charge(i)*mol.atom_charge(j)) * (r1-r2) / r**3
    if atmlst is not None:
        gs = gs[atmlst]
    return gs

In [63]:
def alc_differential_grad_nuc(mol,dL):  # to get the exact diffeential after alch. perturbation
    gn = np.zeros((mol.natm,3))
    for j in range(mol.natm):
        q2 =  mol.atom_charge(j) + dL[j]
        r2 = mol.atom_coord(j) 
        for i in range(mol.natm):
            if i != j:
                q1 = mol.atom_charge(i) +dL[i]
                r1 = mol.atom_coord(i)
                r = np.linalg.norm(r1-r2)
                gn[j] = (q1 * q2-mol.atom_charge(i)*mol.atom_charge(j)) * (r1-r2) / r**3
    return gn

In [68]:
def alc_deriv_grad_nuc(mol,dL):  # to get the derivative with respect to alch. perturbation
    gn = np.zeros((mol.natm,3))
    for j in range(mol.natm):
        q2 =  mol.atom_charge(j)
        r2 = mol.atom_coord(j) 
        for i in range(mol.natm):
            if i != j:
                q1 = mol.atom_charge(i)
                r1 = mol.atom_coord(i)
                r = np.linalg.norm(r1-r2)
                gn[j] = (q1 * +dL[j]+ q2* +dL[i]) * (r1-r2) / r**3
    return gn

In [58]:
a=np.asarray([1,2])
b=np.asarray([3,3])
print(np.sqrt(np.dot(a-b,a-b)))
np.linalg.norm(a-b)

2.23606797749979


2.23606797749979

In [72]:
alc_deriv_grad_nuc(mol0,[.001,-.001])

array([[ 0.        ,  0.        ,  0.00061728],
       [ 0.        ,  0.        , -0.00061728]])

In [73]:
alc_differential_grad_nuc(mol0,[.001,-.001])

array([[ 0.        ,  0.        ,  0.00061698],
       [ 0.        ,  0.        , -0.00061698]])

In [65]:
g1.grad_nuc()-g0.grad_nuc()

array([[ 0.        ,  0.        ,  0.00061698],
       [ 0.        ,  0.        , -0.00061698]])

In [66]:
g0.grad_nuc()

array([[  0.        ,   0.        ,  14.81481481],
       [  0.        ,   0.        , -14.81481481]])

## Final comparison 

In [43]:
ga_1+ga_2+ga_3+ga_4

array([[-1.32153405e-19,  1.15357759e-18, -2.03175016e-04],
       [-3.34375045e-19, -4.31366558e-18,  2.03177912e-04]])

In [44]:
gfd=g1.grad()-g0.grad()

--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000     1.0869705326
1 O    -0.0000000000    -0.0000000000    -1.0869705326
----------------------------------------------
--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0000000000     0.0000000000     1.0871732099
1 O     0.0000000000    -0.0000000000    -1.0871732099
----------------------------------------------


In [45]:
mol_nn=FcM(fcs=[1,-1],atom="C 0 0 0; O 0 0 2.05",unit="Bohr",basis="STO-3G")
mf_nn=scf.RHF(mol_nn)
mf_nn.scf(dm0=mf_nn.init_guess_by_1e())
g_nn=mf_nn.Gradients()
g_nn.run()

converged SCF energy = -105.439066087168
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000     0.1755432988
1 O    -0.0000000000     0.0000000000    -0.1755432988
----------------------------------------------


In [46]:
mol_bf=FcM(fcs=[-1,1],atom="C 0 0 0; O 0 0 2.05",unit="Bohr",basis="STO-3G")
mf_bf=scf.RHF(mol_bf)
mf_bf.scf(dm0=mf_bf.init_guess_by_1e())
g_bf=mf_bf.Gradients()
g_bf.run()

converged SCF energy = -119.891952783638
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000     0.5777073343
1 O    -0.0000000000     0.0000000000    -0.5777073343
----------------------------------------------


In [47]:
mol_bf=FcM(fcs=[0,0],atom="C 0 0 0; O 0 0 2.05",unit="Bohr",basis="STO-3G")
mf_bf=scf.RHF(mol_bf)
mf_bf.scf(dm0=mf_bf.init_guess_by_1e())
g_bf=mf_bf.Gradients()
g_bf.run()

converged SCF energy = -111.21367962675
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000     0.2188666669
1 O    -0.0000000000     0.0000000000    -0.2188666669
----------------------------------------------
